In [ ]:
!pip install yfinance
import math
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout,Bidirectional,GRU 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 111.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 18.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
  

In [ ]:
import pandas as pd
import yfinance as yf

# define ticker symbols
tickers = ['^NSEI', '^GSPC', '^N225', '^GDAXI', '^FTSE']

# create an empty DataFrame to store the stock data
stock_data = pd.DataFrame()

# loop over each ticker and extract the stock data
for ticker in tickers:
    stock = yf.download(ticker, start='2013-01-01', end='2022-01-01')
    stock = stock[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    stock.columns = [f"{ticker}_{col}" for col in stock.columns]
    stock_data = pd.concat([stock_data, stock], axis=1)

# fill missing values with the data just before in the column
# stock_data.fillna(method='ffill', inplace=True)

stock_data = stock_data.interpolate(method='cubic', linear = True)


# reset index
stock_data.reset_index(inplace=True)

# rename date column
stock_data = stock_data.rename(columns={'Date': 'date'})


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
stock_data.shape

(2343, 31)

In [ ]:
stock_data.tail(10)

,date,^NSEI_Open,^NSEI_High,^NSEI_Low,^NSEI_Close,^NSEI_Adj Close,^NSEI_Volume,^GSPC_Open,^GSPC_High,^GSPC_Low,...,^GDAXI_Low,^GDAXI_Close,^GDAXI_Adj Close,^GDAXI_Volume,^FTSE_Open,^FTSE_High,^FTSE_Low,^FTSE_Close,^FTSE_Adj Close,^FTSE_Volume
2333,2021-12-20,16824.250000,16840.099609,16410.199219,16614.199219,16614.199219,330600.0,4587.899902,4587.899902,4531.100098,...,15060.099609,15239.669922,15239.669922,6.911880e+07,7269.899902,7269.899902,7101.000000,7198.000000,7198.000000,9.250622e+08
2334,2021-12-21,16773.150391,16936.400391,16688.250000,16770.849609,16770.849609,241000.0,4594.959961,4651.140137,4583.160156,...,15305.610352,15447.440430,15447.440430,5.332970e+07,7198.000000,7297.399902,7198.000000,7297.399902,7297.399902,6.836697e+08
2335,2021-12-22,16865.550781,16971.000000,16819.500000,16955.449219,16955.449219,207900.0,4650.359863,4697.669922,4645.529785,...,15425.160156,15593.469727,15593.469727,4.299660e+07,7297.399902,7341.700195,7273.500000,7341.700195,7341.700195,5.147729e+08
2336,2021-12-23,17066.800781,17118.650391,17015.550781,17072.599609,17072.599609,197000.0,4703.959961,4740.740234,4703.959961,...,15622.599609,15756.309570,15756.309570,4.325390e+07,7341.700195,7385.399902,7331.100098,7373.299805,7373.299805,4.226715e+08
2337,2021-12-24,17149.500000,17155.599609,16909.599609,17003.750000,17003.750000,182600.0,4716.381324,4766.087566,4724.145462,...,15706.669836,15812.663752,15812.663752,4.084918e+07,7373.299805,7403.700195,7368.799805,7372.100098,7372.100098,1.260055e+08
2338,2021-12-27,16937.750000,17112.050781,16833.199219,17086.250000,17086.250000,144800.0,4733.990234,4791.490234,4733.990234,...,15697.440430,15835.250000,15835.250000,2.889960e+07,7352.298472,7456.523195,7359.153059,7408.712634,7408.712634,2.844441e+08
2339,2021-12-28,17177.599609,17250.250000,17161.150391,17233.250000,17233.250000,176000.0,4795.490234,4807.020020,4780.040039,...,15832.959961,15963.700195,15963.700195,3.402100e+07,7347.709398,7463.994157,7354.945441,7421.470803,7421.470803,4.388919e+08
2340,2021-12-29,17220.099609,17285.949219,17176.650391,17213.599609,17213.599609,161700.0,4788.640137,4804.060059,4778.080078,...,15803.589844,15852.250000,15852.250000,3.497390e+07,7372.100098,7457.100098,7372.100098,7420.700195,7420.700195,4.459757e+08
2341,2021-12-30,17201.449219,17264.050781,17146.349609,17203.949219,17203.949219,320800.0,4794.229980,4808.930176,4775.330078,...,15834.570312,15884.860352,15884.860352,3.245660e+07,7420.700195,7433.399902,7403.000000,7403.000000,7403.000000,2.664027e+08
2342,2021-12-31,17244.500000,17400.800781,17238.500000,17354.050781,17354.050781,167000.0,4775.209961,4786.830078,4765.750000,...,NaN,NaN,NaN,NaN,7403.000000,7403.000000,7357.100098,7384.500000,7384.500000,1.793438e+08


In [ ]:
missing_values = stock_data.isnull().sum() 
print(missing_values)

date                0
^NSEI_Open          0
^NSEI_High          0
^NSEI_Low           0
^NSEI_Close         0
^NSEI_Adj Close     0
^NSEI_Volume        0
^GSPC_Open          0
^GSPC_High          0
^GSPC_Low           0
^GSPC_Close         0
^GSPC_Adj Close     0
^GSPC_Volume        0
^N225_Open          2
^N225_High          2
^N225_Low           2
^N225_Close         2
^N225_Adj Close     2
^N225_Volume        2
^GDAXI_Open         0
^GDAXI_High         0
^GDAXI_Low          0
^GDAXI_Close        0
^GDAXI_Adj Close    0
^GDAXI_Volume       0
^FTSE_Open          0
^FTSE_High          0
^FTSE_Low           0
^FTSE_Close         0
^FTSE_Adj Close     0
^FTSE_Volume        0
dtype: int64
